## **Step 1 - keywords Extraction**
***

We have two datasets, one with dream text descriptions:

In [16]:
from keyword_extractor import read_datasets, extract_and_save_keywords_from_dataframes
from yaml_parser import load_config
config = load_config()
dream_df, keywords_df = read_datasets(config)
dream_df.head()

,dream_id,dreamer,description,dream_date,dream_language,text_dream,characters_code,emotions_code,aggression_code,friendliness_code,...,Animal,Friends,Family,Dead&Imaginary,Aggression/Friendliness,A/CIndex,F/CIndex,S/CIndex,NegativeEmotions,Dream Symbol
0,1,alta,Alta: a detailed dreamer,1957,en,"The one at the Meads's house, where it's bigge...","2ISA, 1MKA, 1FDA, 1IOA, 2ISA",NaN,2IKA > Q,2IKA 4> Q,...,0.000000,0.200000,0.200000,0.0,0.000,0.200000,0.200000,0.0,0.0,"Upstairs, Haunted\nHouse, Maid, Tea\nHouse, Ma..."
1,2,alta,Alta: a detailed dreamer,8/11/1967,en,I'm at a family reunion in a large fine house ...,"2ISA, people, 2ISA",SD 2IKA,"D > Q, Q > 2ISA",NaN,...,0.000000,0.000000,0.000000,0.0,1.000,0.666667,0.000000,0.0,1.0,"Fainting, Near\nDeath Experience, Landslide, P..."
2,3,alta,Alta: a detailed dreamer,8/1/1985,en,I watch a plane fly past and shortly realize i...,"2ISA, 2ISA, 1FSA, 1MBA, 1IOA, 2ISA, 2FDA","SD 1ISA, AP D, AP D","It PRP >, It PRP >, D > 1FKA",NaN,...,0.000000,0.000000,0.285714,0.0,1.000,0.428571,0.000000,0.0,1.0,"Kidnap, Emergency\nAlert, Death\nPenalty, Abor..."
3,4,alta,Alta: a detailed dreamer,1985?,en,Me pulling the green leaves and berries off so...,"1MAA, 1FMA, 2ISA, 2IKA, 1ANI, 1ANI, 1IOA, 2ISA...","SD 2ISA, SD D","Q > Q, 2ISA > Q, 2ISA > Q, D > 1MSA","1IKA 4> Q, 2ISA 4> 2ISA",...,0.176471,0.142857,0.142857,0.0,1.000,0.235294,0.117647,0.0,1.0,"Kidnap, Knocking, Party, Sabotage, Paranormal"
4,5,alta,Alta: a detailed dreamer,1985?,en,I'm in a room that reminds me of (but definite...,"1IRA, 1MSA, 1ISA, 2ISA, 1ISA, 1IKA","AP D, AP D, AP 1MSA, CO D, SD D, AP D","1MSA > D, Q > Q, D > 2IKA, D > 2IKA, D > 1MSA,...",D 4> Q,...,0.000000,0.166667,0.166667,0.0,0.875,1.333333,0.166667,0.0,1.0,"Haunted\nHouse, Paranormal, Near\nDeath Experi..."


And another one with interpretations of dreams according to keywords:

In [17]:
keywords_df.head()

,Dream Symbol,Interpretation
0,Aardvark,To see an aardvark in your dream indicates tha...
1,Abandonment,To dream that you are abandoned suggests that ...
2,Abduction,To dream of being abducted indicates that you ...
3,Aborigine,To see an Aborigine in your dream represents b...
4,Abortion,To dream that you have an abortion suggests th...


Now, we will use pretrained LLMs in order to extract the given keywords from the keywords dataset , from the dream text description from the dream text dataset.

### **all-MiniLM-L6-v2**
***

In [18]:
extract_and_save_keywords_from_dataframes()

In [19]:
# watch extracted keywords
dream_df, keywords_df = read_datasets(config)
columns_to_show = [config['data']['dream_text_column'], config['data']['keywords_column']]
dream_df[columns_to_show].head()

,text_dream,Dream Symbol
0,"The one at the Meads's house, where it's bigge...","Upstairs, Haunted\nHouse, Maid, Tea\nHouse, Ma..."
1,I'm at a family reunion in a large fine house ...,"Fainting, Near\nDeath Experience, Landslide, P..."
2,I watch a plane fly past and shortly realize i...,"Kidnap, Emergency\nAlert, Death\nPenalty, Abor..."
3,Me pulling the green leaves and berries off so...,"Kidnap, Knocking, Party, Sabotage, Paranormal"
4,I'm in a room that reminds me of (but definite...,"Haunted\nHouse, Paranormal, Near\nDeath Experi..."
